In [1]:
# Set the range for mixing parameters
import numpy as np

step_total  = 10
step_size = 0.05
MU = np.around(np.arange(step_size, step_size * step_total + 0.01, step_size), decimals=2)


# Specify Parameters
n = 1000
tau1 = 2  # Power-law exponent for the degree distribution
tau2 = 1.1  # Power-law exponent for the community size distribution
#mu = 0.1  # Mixing parameter
avg_deg = 25  # Average Degree
max_deg = int(0.1 * n)  # Max Degree
min_commu = 60  # Min Community Size
max_commu = int(0.1 * n)  # Max Community Size

sample_count = 20

In [2]:
import os
import pickle
import networkx as nx
from networkx.generators.community import LFR_benchmark_graph  # Replace with your actual import

def gene_lfr_graph_mu(mu, sample_count=sample_count, output_dir='data'):
    # Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    graphs = []
    memberships = []

    for i in range(sample_count):
        G = LFR_benchmark_graph(
            n, tau1, tau2, mu, average_degree=avg_deg, max_degree=max_deg,
            min_community=min_commu, max_community=max_commu,
            seed=2024
        )

        G = nx.Graph(G)
        selfloop_edges = list(nx.selfloop_edges(G))
        G.remove_edges_from(selfloop_edges)

        intrinsic_communities = {frozenset(G.nodes[v]["community"]) for v in G}
        intrinsic_membership = {}
        for node in range(G.number_of_nodes()):
            for index, inner_set in enumerate(intrinsic_communities):
                if node in inner_set:
                    intrinsic_membership[node] = index
                    break
        intrinsic_membership = list(intrinsic_membership.values())

        # Append the graph and membership to the lists
        graphs.append(G)
        memberships.append(intrinsic_membership)

    # Save all graphs and memberships into a single file
    combined_data = {
        'graphs': graphs,
        'memberships': memberships
    }
    
    file_path = os.path.join(output_dir, f'lfr_graph_mu{mu}.pickle')
    with open(file_path, 'wb') as file:
        pickle.dump(combined_data, file)

    print(f'Saved all graphs and memberships for mu={mu} to {file_path}')

In [3]:
from concurrent.futures import ProcessPoolExecutor

with ProcessPoolExecutor() as executor:
    results = list(executor.map(gene_lfr_graph_mu, MU))

Saved all graphs and memberships for mu=0.5 to data/lfr_graph_mu0.5.pickle
Saved all graphs and memberships for mu=0.45 to data/lfr_graph_mu0.45.pickle
Saved all graphs and memberships for mu=0.4 to data/lfr_graph_mu0.4.pickle
Saved all graphs and memberships for mu=0.35 to data/lfr_graph_mu0.35.pickle
Saved all graphs and memberships for mu=0.3 to data/lfr_graph_mu0.3.pickle
Saved all graphs and memberships for mu=0.25 to data/lfr_graph_mu0.25.pickle
Saved all graphs and memberships for mu=0.2 to data/lfr_graph_mu0.2.pickle
Saved all graphs and memberships for mu=0.15 to data/lfr_graph_mu0.15.pickle
Saved all graphs and memberships for mu=0.1 to data/lfr_graph_mu0.1.pickle
Saved all graphs and memberships for mu=0.05 to data/lfr_graph_mu0.05.pickle


In [4]:
def load_lfr_graph_mu(mu, output_dir='data'):
    file_path = os.path.join(output_dir, f'lfr_graph_mu{mu}.pickle')
    with open(file_path, 'rb') as file:
        combined_data = pickle.load(file)
    
    graphs = combined_data['graphs']
    memberships = combined_data['memberships']
    
    return graphs, memberships

# Example usage
# graphs, memberships = load_data(mu_value)
graphs, memberships = load_lfr_graph_mu(mu=0.2)